# Class 7 - Metaprogramming

Today we'll cover metaprogramming in Julia.  This should answer some questions about what's really going on when you use a macro such as `@simd` or a expression such as `:hello`.  

# What is Metaprogramming?

Briefly, [metaprogramming](https://en.wikipedia.org/wiki/Metaprogramming) refers to writing programs to write programs.  This is poweful because it allows you to write less code for repetitive tasks, and even write domain-specific languages.  [Template Metaprogramming](https://en.wikipedia.org/wiki/Template_metaprogramming) is one example of this.  e.g.

In [ ]:
function cme257fn{T}(a::T, b::T)
    a + T(2)*b
end
;

In [ ]:
1+1

allows you to write a function for any number of types `T`.  We've already seen quite a bit of this, and today we'll talk more about macros and expressions.

## Reference

Today we'll mostly cover material that can be found in the [metaprogramming](https://docs.julialang.org/en/stable/manual/metaprogramming/) section of Julia's documentation.  Many examples will be drawn from the documentation.

# Expressions - How Text Becomes a Program

Everything you put into a Jupyter notebook, a `.jl` file, or the Julia REPL starts as a string.

In [ ]:
prog = "1 + 1"
typeof(prog)

Julia then parses this string to determine what to do with the input

In [ ]:
exp = parse(prog)

In [ ]:
typeof(exp)

You can think of expressions as "quoting" code, not evaluating it.  For example `1+1` evalueates to 2, but the expression `:(1+1)` is akin to the satement `'1+1' is an equation`, which doesn't refer to the result of the evaluation, but the statement itself

In [ ]:
# expressions have a symbol that denotes the type of expression
@show typeof(exp.head)
exp.head

In [ ]:
# expression arguments may be symbols, other expressions, or literal values
exp.args

This means that the program itself is stored in a data structure accessible through Julia!

You can always run your program using `eval`

In [ ]:
@show exp
eval(exp)

Some useful tools for working with expressions are `dump` and `Meta.show_sexpr`.  These are both ways of visualizing the [abstract syntax tree (AST)](https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming#The_Abstract_Syntax_Tree) that is generated when code is parsed.

In [ ]:
# more detailed printout of expression
dump(exp)

In [ ]:
exp = parse("(1+1)/2")

In [ ]:
# s-expressions represent expression as tree
Meta.show_sexpr(exp)

In [ ]:
dump(exp)

# Symbols, Quoting

Symbols are denoted by the colon `:` symbol.  These are [interned strings](https://en.wikipedia.org/wiki/String_interning) which is used to build expressions.  

In [ ]:
@show typeof(:hello)
@show typeof(Symbol("hello"))
@show sym = Symbol("hello", "_world", 10)
@show typeof(sym)
;

A second role for the `:` character is for "quoting", which helps build expressions


In [ ]:
exp = :(1 + 2 * 3)
dump(exp)

quoing can also be done using the `quote` keyword

In [ ]:
exp = quote
    x = 1
    x + x
end
@show typeof(exp)
exp

In [ ]:
dump(exp)

In [ ]:
exp1 = :(2+3 )
exp2 = Expr(:call, :+, 1, exp1)

In [ ]:
exp2 = :(1 + (2 + 3))
call(1 + exp1)

# Exercise 1

1. try putting a simple for-loop in a quote.  What does it look like? Try using `dump` and `Meta.show_sexpr`
2. Create an expression for `1 + (2+3)` starting with the expression `:(2+3)`
3. use quoting with `:` to examine the expression for the function `sum257{T}(a::T,b::T) = a+b`.  Try rewriting the function using the `function` keyword, and creating an expression using a `quote` block.

# Interpolation

Just like you can interpolate in expressions by using `$`, just like you can in strings

In [ ]:
x = 1
"x = $x"

In [ ]:
a = 1
exp = :($a + b)

In [ ]:
# you can also interpolate expressions.
exp1 = :(1+1)
exp2 = :($exp1/2)

# Functions of Expressions

Since expressions are a data type in Julia, you can create functions of expresssions

In [ ]:
:(1 + 1)
bin_exp(:+, 1, 1)

In [ ]:
# expression for binary operation
function bin_exp(op, arg1, arg2)
   return Expr(:call, op, arg1, arg2) 
end

exp = bin_exp(:+, 1,2)
@show exp
@show eval(exp)

exp = bin_exp(:*, 2, :(1+1))
exp2 = bin_exp(:*, 2, bin_exp(:+,1,1))
@show exp
@show exp == exp2
@show eval(exp)
;

# Macros

Macros give you a way to generate code to be included in a program.  They are evaluated directly, and do not require a call to `eval()`.  They are created using the `macro` keyword.

In [ ]:
macro helloworld()
    return :( println("Hello World!") )
end

In [ ]:
@helloworld

In [ ]:
# macros can take arguments
macro hello(name)
    return :(println("Hello ", $name))
end

In [ ]:
@hello "CME 257"

In [ ]:
# you can also call a macro with function-like syntax
@hello("CME 257")

There are tools to help you build, understand, and debug macros - the function `macroexpand()` and the macro `@macroexpand`

In [ ]:
macroexpand(:(@hello("CME 257")))

In [ ]:
# the macro version is easier to call
@macroexpand @hello "CME 257"

Macros are executed at parse time, not at run time (or compile time).  This means they don't add overhead to a function call (since they generate actual code).  However, they may add overhead to using a module, or including a file.  

In [ ]:
macro twostep(arg)
   println("I execute at parse time. The argument is: ", arg)
   return :(println("I execute at runtime. The argument is: ", $arg))
end

ex = @macroexpand @twostep 1+1

In [ ]:
eval(ex)

## Examples

We've already seen the `@assert` macro

In [ ]:
# does nothing
@assert 1==1
# throws error
@assert 1==2
;

The [following example](https://docs.julialang.org/en/stable/manual/metaprogramming/#Building-an-advanced-macro-1) is given as a way to implement the `@assert` macro

In [ ]:
macro assert2(ex)
   return :( $ex ? nothing : throw(AssertionError($(string(ex)))) )
end
# does nothing
@assert2 1==1
# does something
@assert2 1==2
;

if you do something more complicated in a macro, you may wish to use local variables to avoid creating new variables outside the macro.  Note you can use the `local` keyword in more general contexts.

In [ ]:
@time 1+1

In [ ]:
macro time2(ex)
    return quote
        local t0 = time()
        local val = $ex
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

In [ ]:
a = 1
b = 2
@time2 a + b

In [ ]:
t0 = 1
t1 = 2
@time2 t0 + t0

We see that even though we tried to avoid a name conflict, we still ran into trouble because we interpolated the expression in the macro definition.  We can avoid this by escaping the expression with `esc()`

In [ ]:
macro time2(ex)
    return quote
        local t0 = time()
        local val = $(esc(ex))
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

In [ ]:
t0 = 1
t1 = 2
@time2 t0+t0

In [ ]:
macro show2(ex)
    local name = string(ex)
    local val =  eval(ex)
    println(string(name, " = ", val))   
    val
end

In [ ]:
@show2 x

# Exercise 2

1. Create a macro `@show2` that does the same thing as the `@show` macro
2. compare expressions generated in `@assert` and `@assert2` with the same input. You'll likely want to use `@macroexpand` and `dump()`
3. compare the expressions generated by `@show` and your `@show2` macro with the same input.

# Code Generation

Metaprogramming can save you quite a bit of time and effort when it comes to writing repetitive code

In [ ]:
import Base: +, -, *, /
struct cme257type{T}
    val::T
end

for op in (:+, :-, :*, :/)
   eval(quote
        ($op){T}(a::cme257type{T}, b::cme257type{T}) = cme257type{T}($(op)(a.val, b.val))
    end)
end

In [ ]:
+{T}(a::cme257type{T}, b::cme257type{T}) = cme257type{T}(+(a.val, b.val))

In [ ]:
a = cme257type(1)
b = cme257type(2)
a * b

there's a handy `@eval` macro that can be used to do the same thing

In [ ]:
# option 1
for op in (:+, :-, :*, :/)
   @eval begin
        ($op){T}(a::cme257type{T}, b::cme257type{T}) = cme257type{T}($(op)(a.val, b.val))
    end
end

# option 2
for op in (:+, :-, :*, :/)
   @eval ($op){T}(a::cme257type{T}, b::cme257type{T}) = cme257type{T}($(op)(a.val, b.val))
end

In [ ]:
a = cme257type(1.0)
b = cme257type(2.0)
a / b

# Generated Functions

Generated functions are a powerful way of creating functions, based on the types of the arguments.  Note that you have to be careful to avoid unintended side effects with them, so refer to the [documentation](https://docs.julialang.org/en/stable/manual/metaprogramming/#Generated-functions-1) if you indend to use them seriously, beyond this brief introduction.

In [ ]:
@generated function foo(x)
   Core.println(x)
   return :(x * x)
end

In [ ]:
foo(x) = x*x

In [ ]:
x = foo(2.0)
;

note that the generated function is generally only produced once. If you run this cell twice with the same type of input, you probably won't see the type output again.  However, there's no guarantee that this will happen, which is why the functions can't have any side effects 

In [ ]:
x

For the next example, note that you can create a function with variable length arguments using `...`

In [ ]:
function test_fn(I::Int64...)
    println("n inputs = ", length(I))
    for i = 1:length(I)
       println("  $i: $(I[i])") 
    end
end

In [ ]:
test_fn(1, 5, 3)

Now, for the example, which is described in [the metaprogramming documentaion](https://docs.julialang.org/en/stable/manual/metaprogramming/#An-advanced-example-1).  Recall that in Julia, arrays are stored in column-major format.  There is a function `sub2ind()` that will transform indices of an array into a single array index.

In [ ]:
A = [1 2;3 4]
# two ways of accessing an array: single index and multi-index
@show A[2]
@show A[2,1]
# the following gives us the conversion from the multi-index to the single index
@show sub2ind((2,2), 2, 2)
A

For a 2-dimensional array `A` with dimensions `(m,n)`, the formula for indices `A[i,j]` is `i + m*(j-1)`.  We'll see how this generalizes shortly.

Now, the clever use of a generated function: we can write a single generated function that will produce the output of `sub2ind` that dispatches based on the number of dimensions.

In [ ]:
@generated function sub2ind_gen(dims::NTuple{N}, I::Integer...) where N
   length(I) == N || return :(error("partial indexing is unsupported"))
   ex = :(I[$N] - 1)
   for i = (N - 1):-1:1
       ex = :(I[$i] - 1 + dims[$i] * $ex)
   end
   return :($ex + 1)
end

In [ ]:
sub2ind_gen((2,2), 2, 2)

Note that we could do something similar with a non-generated function, but then we would evaluate the loop at run-time, instead of complie time.  Since there is some overhead associated with loops, for frequently called functions this can give performance benefits.

In [ ]:
function sub2ind_loop(dims::NTuple{N}, I::Integer...) where N
    length(I) == N || return error("partial indexing is unsupported")
    ind = I[N] - 1
    for i = N-1:-1:1
        ind = I[i]-1 + dims[i]*ind
    end
    return ind + 1
end


# Exercise 3

Recall how we were able to call library functions e.g.  `ccall((:cos, "libm"), Float64, (Float64,), 1.0)`. Generate a set of wrappers for the following functions in libm - `cos, sin, acos, asin, tan, fabs, cosh` (just work with `Float64` inputs).  To avoid conflicts with existing funcitons, name the wrappers `cme257op` where `op` is the operation name.  Add some more functions with a single `Float64` input from [here](https://en.wikipedia.org/wiki/C_mathematical_functions)

Hint: you can quote an expression `ex` using `Expr(:quote, ex)` - this is useful for the first argument of `ccall`

If you have time, check out how TensorFlow.jl [wraps the TensorFlow library](https://github.com/malmaud/TensorFlow.jl/blob/master/src/ops/math.jl) using metaprogramming

# Further Reading

* [Wikibook on Metaprogramming in Julia](https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming)
* [On Machine Learning and Programming Languages](https://julialang.org/blog/2017/12/ml&pl)
* Check out how TensorFlow.jl [wraps the TensorFlow library](https://github.com/malmaud/TensorFlow.jl/blob/master/src/ops/math.jl)

In [ ]:
# Exercise 3 example solution
ops = (:cos, :sin, :acos, :asin, :tan, :fabs, :cosh)
for op in ops
    fname = Symbol(:cme257,op)
    println("generating $fname")
    opn = Expr(:quote, op)
    @eval begin
        $(fname)(x::Float64) = ccall(($opn, "libm"), Float64, (Float64,), x)
    end
end